In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch
import trimesh
from trimesh import voxel
from functools import partial
import ssl
import os
import pickle
from config import cfg
import torch.optim as optim
import nrrd
import random
from Script.scripts import augment_voxel_tensor,sample_z,vizualizer_stanford_data


In [2]:
pp=torch.rand([2,2])

In [6]:
torch.sum(torch.exp(pp),1)

tensor([3.6479, 3.8791])

In [2]:
from DataLoader.GANloader import GANLoader
from DataLoader.GanDataGen import GANDataGenerator
from Models.Generator import Generator32,Generator32_Small
from Models.Discriminator import Discriminator32,Discriminator32_Small

from Solvers.SolverGAN import SolverGAN
device=cfg.DEVICE
#print(device)
PYTORCH_ENABLE_MPS_FALLBACK=1
#for mac os fix 
ssl._create_default_https_context = ssl._create_unverified_context

torch.manual_seed(cfg.SEED)
np.random.seed(cfg.SEED)

generator=Generator32_Small()
generator.load_state_dict(torch.load(os.path.join('./../SavedModels',"generator_model.pth")))
generator.eval()




Generator32_Small(
  (fc1): Sequential(
    (0): Linear(in_features=160, out_features=8192, bias=True)
    (1): BatchNorm1d(8192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): Sequential(
    (0): ConvTranspose3d(128, 128, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
    (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): Sequential(
    (0): ConvTranspose3d(128, 64, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv5): Sequential(
    (0): ConvTranspose3d(64, 1, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
  )
)

In [3]:
with open(getattr(cfg,"GAN_TRAIN_SPLIT"), 'rb') as pickle_file:
    tmp=pickle.load(pickle_file)

In [4]:
def loadOneElem(idx,data):

    model_id=data[idx][0]
    learned_embedding = data[idx][1]

    
    voxel,_=nrrd.read(os.path.join(cfg.GAN_VOXEL_FOLDER,model_id,model_id+'.nrrd'))
    voxel = torch.FloatTensor(voxel)
    voxel /=255.

    learned_embedding=torch.Tensor(learned_embedding)
    learned_embedding=torch.cat((learned_embedding.unsqueeze(0),sample_z()),1).squeeze(0)


    return model_id,learned_embedding,voxel

In [10]:
idx=random.randint(0,len(tmp))

k=loadOneElem(idx,tmp)


In [6]:
def vizualizer_stanford_data(vox_color,data):

    if data=='fake':
        condition1 = vox_color[0, :] < 0.5
        condition2 = vox_color[0, :] >= 0.5
        

        vox_color[0,condition1] = 0.0

        vox_color[0,condition2] = 1.0

        print(vox_color.shape)
        vox_color=vox_color.expand(4, -1, -1, -1)
    

    vox_color=np.array(vox_color)
    vox_color = np.rollaxis(vox_color, 0,4)
    ind=vox_color[:,:,:]!=[0,0,0,0]
    vox = ind[:, :, :,0]
    z=voxel.VoxelGrid(vox)
    l=z.as_boxes()
    return l.show()


In [11]:
p=generator(k[1].unsqueeze(0))
p=p['sigmoid_output'].squeeze(0).detach()

In [8]:
def custom_replace(tensor):
    # we create a copy of the original tensor, 
    # because of the way we are replacing them.
    res = tensor.clone()
    res[tensor>=0.5] = 1
    res[tensor<0.5] = 0
    return res

In [13]:
vizualizer_stanford_data(p,'fake')

torch.Size([1, 32, 32, 32])


In [12]:
vizualizer_stanford_data(k[2],'true')